In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()


(6759, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,7.420711,1.850026,2.083403,0.116689,0.116689,1.8857,0.116689
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,7.0,1.9587,0.0824,6.791545,1.809917,2.003431,0.096757,0.096757,1.9587,0.082400
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,7.222071,1.816207,2.099822,0.141808,0.141808,1.7824,0.141808
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,7.0,1.7963,0.0565,6.505044,1.796547,1.991641,0.097547,0.097547,1.7963,0.067659
4,4,2,Göteborg Majorna OK 1,Göteborg Majorna OK,SWE,1,6.0,amanda hansson,1,Amanda Hansson,2.0,2.0053,0.0358,6.581146,1.852547,2.024809,0.086131,0.086131,2.0053,0.086131


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std,num_runs
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.116689,1.0
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9587,0.082400,7.0
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.141808,1.0
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.7963,0.067659,7.0
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.086131,2.0


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                              ... log_mean           log_std            \
leg                        2  ...        3       4         1         2   
team_id                       ...                                        
1           Magdalena Olsson  ...   1.7824  1.7963  0.116689  0.082400   
2              Runa Fremstad  ...   1.9581  1.9854  0.086131  0.087900   
3             Anni Heikkonen  ...   1.8669  1.9113  0.067659  0.100600   
4        Anna Dahlgren-Rosén  ...   1.8086  1.9289  0.112800  0.182322   
5            Josefine Heikka  ...   1.9000  1.8720  0.102900  0.094094   

                            num_runs                 
leg             3         4        1    2    3    4  
team_id                                              
1        0.141808  0.067659      1.0  7.0  1.0  7.0  
2        0.067659  0.093119      2.0  5.0  5.0  3.0  
3        0.135916  0.067659      4.0  7.0  1.0  8.0  
4        0.130028  0.101968      4.0  0.0  1.0  2.0  
5        0.095300  0.119901      7.0  2.0  6.0  3.0  

[5 rows x 24 columns]

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 24)

(1682, 24)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,31.362034,69.773084,108.485814,154.942833,49.818220,92.900628,141.776324,190.800881,39.411989,80.083961,...,32.740572,41.667187,49.818220,47.549226,56.843543,54.381271,39.411989,40.474712,43.408851,47.623576
2,37.518297,77.206722,127.003574,181.134030,52.816954,98.603649,152.246951,213.941742,44.572995,87.190284,...,45.168913,47.843606,52.816954,50.385737,58.967162,68.947799,44.572995,42.571131,51.649722,57.552342
3,34.805132,70.436801,112.248206,164.566367,45.365477,89.961411,144.429311,199.558281,39.719039,79.609639,...,36.246272,46.977235,45.365477,48.522304,61.678828,61.060222,39.719039,39.740944,47.233893,53.379527
4,34.481703,69.324147,110.996231,162.338903,53.948813,105.563013,153.516198,210.964759,43.369007,85.365592,...,34.331117,44.347374,53.948813,59.075002,57.575415,66.364641,43.369007,41.718864,44.510994,54.430385
5,32.201035,67.443935,113.028365,159.965271,48.345844,88.847561,141.628550,198.064042,39.754886,77.612487,...,40.424666,40.591304,48.345844,45.234091,58.982589,65.017475,39.754886,37.692754,48.758764,51.260796


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4,num_runs_1,num_runs_2,num_runs_3,num_runs_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,1.78,1.80,0.12,0.08,0.14,0.07,1.0,7.0,1.0,7.0
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,1.96,1.99,0.09,0.09,0.07,0.09,2.0,5.0,5.0,3.0
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,1.87,1.91,0.07,0.10,0.14,0.07,4.0,7.0,1.0,8.0
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,1.81,1.93,0.11,0.18,0.13,0.10,4.0,0.0,1.0,2.0
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,1.90,1.87,0.10,0.09,0.10,0.12,7.0,2.0,6.0,3.0


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,32.740572,41.667187,49.818220,47.549226,56.843543,54.381271,39.411989,40.474712,43.408851,47.623576
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.168913,47.843606,52.816954,50.385737,58.967162,68.947799,44.572995,42.571131,51.649722,57.552342
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,36.246272,46.977235,45.365477,48.522304,61.678828,61.060222,39.719039,39.740944,47.233893,53.379527
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.331117,44.347374,53.948813,59.075002,57.575415,66.364641,43.369007,41.718864,44.510994,54.430385
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.424666,40.591304,48.345844,45.234091,58.982589,65.017475,39.754886,37.692754,48.758764,51.260796


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.116689       1.0    31.362034   49.818220   39.411989   
        2    0.082400       7.0    69.773084   92.900628   80.083961   
        3    0.141808       1.0   108.485814  141.776324  123.776830   
        4    0.067659       7.0   154.942833  190.800881  171.526465   
2       1    0.086131       2.0    37.518297   52.816954   44.572995   
        2    0.087900       5.0    77.206722   98.603649   87.190284   
        3    0.067659       5.0   127.003574  152.246951  139.123727   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
1       1     39.743442          3.675452         0.118180         0.0   
        2     80.327724          4.383499         0.072250         0.0   
        3    124.132686          4.819016         0.068305         0.0   
        4    171.847812          5.145210         0.052867         0.0   
2       1     44.728959          3.796858         0.086747         0.0   
        2     87.410952          4.468692         0.062102         0.0   
        3    139.197476          4.934817         0.046413         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
1       1                0.0         31.362034       49.818220   
        2                0.0         34.335339       47.549226   
        3                0.0         32.740572       56.843543   
        4                0.0         41.667187       54.381271   
2       1                0.0         37.518297       52.816954   
        2                0.0         35.833819       50.385737   
        3                0.0         45.168913       58.967162   

             personal_median  
team_id leg                   
1       1          39.411989  
        2          40.474712  
        3          43.408851  
        4          47.623576  
2       1          44.572995  
        2          42.571131  
        3          51.649722

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [26]:
start_timestamp
pd.Timestamp(start_timestamp).tz_localize(None)

Timestamp('2019-06-15 14:00:00+0300', tz='Europe/Helsinki')

In [27]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]
local_start_ts = pd.Timestamp(start_timestamp).tz_localize(None)

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= local_start_ts)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.116689       1.0    31.362034   49.818220   39.411989   
        2    0.082400       7.0    69.773084   92.900628   80.083961   
        3    0.141808       1.0   108.485814  141.776324  123.776830   
        4    0.067659       7.0   154.942833  190.800881  171.526465   
2       1    0.086131       2.0    37.518297   52.816954   44.572995   

               fin_mean  ...  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg              ...                                                 
1       1     39.743442  ...         0.118180         0.0              0.0   
        2     80.327724  ...         0.072250         0.0              0.0   
        3    124.132686  ...         0.068305         0.0              0.0   
        4    171.847812  ...         0.052867         0.0              0.0   
2       1     44.728959  ...         0.086747         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           31.362034       49.818220        39.411989   
        2           34.335339       47.549226        40.474712   
        3           32.740572       56.843543        43.408851   
        4           41.667187       54.381271        47.623576   
2       1           37.518297       52.816954        44.572995   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:24.719364882 2019-06-15 14:31:21.722012043   
        2   2019-06-15 15:20:05.037651539 2019-06-15 15:09:46.385051966   
        3   2019-06-15 16:03:46.609815836 2019-06-15 15:48:29.148831129   
        4   2019-06-15 16:51:31.587908268 2019-06-15 16:34:56.570000648   
2       1   2019-06-15 14:44:34.379707098 2019-06-15 14:37:31.097831011   

                           fin_time_end95 personal_estimate_interval  
team_id leg                                                           
1       1   2019-06-15 14:49:49.093224049                  18.456187  
        2   2019-06-15 15:32:54.037665844                  13.213887  
        3   2019-06-15 16:21:46.579432011                  24.102971  
        4   2019-06-15 17:10:48.052886248                  12.714084  
2       1   2019-06-15 14:52:49.017218351                  15.298656  

[5 rows x 21 columns]

In [38]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

for_print[for_print['team'].str.contains("Stadin")]
#for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,
1,1,Tilda Östberg,1.0,31.36,39.41,49.82,18.46
1,2,Magdalena Olsson,7.0,34.34,40.47,47.55,13.21
1,3,Julia Jakob,1.0,32.74,43.41,56.84,24.10
1,4,Tove Alexandersson,7.0,41.67,47.62,54.38,12.71
2,1,Amanda Hansson,2.0,37.52,44.57,52.82,15.30
...,...,...,...,...,...,...,...
1722,4,Johanna Liinamaa,3.0,68.73,89.36,116.31,47.59
1723,1,Elina Holstikko,0.0,62.81,89.94,128.60,65.79
1723,2,Mirka Leinonen,0.0,59.52,85.00,121.54,62.02


In [39]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Downt")]

,team_id,leg,team,name,num_runs,ind_log_mean,ind_log_std,personal_start95,personal_median,personal_end95,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,8.0,2.188600,0.126500,41.734514,53.537626,68.417273,3.980630,0.125453,2019-06-15 11:41:44.070834398+00:00,2019-06-15 12:08:25.036398888+00:00,2019-06-15 11:53:32.257576227+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,7.0,2.296600,0.144100,42.708030,56.676541,75.730219,4.707501,0.096904,2019-06-15 12:31:32.412785769+00:00,2019-06-15 13:14:01.803070307+00:00,2019-06-15 12:50:52.509668111+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,5.0,2.334200,0.141700,56.983412,75.177391,99.779027,5.229519,0.081018,2019-06-15 13:39:19.955246210+00:00,2019-06-15 14:39:02.906992197+00:00,2019-06-15 14:06:36.491135359+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,1.0,2.293200,0.171039,56.319545,78.230465,109.492939,5.583356,0.075767,2019-06-15 14:49:25.044133663+00:00,2019-06-15 16:08:27.736457586+00:00,2019-06-15 15:25:42.472955465+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,3.0,2.361600,0.166801,45.734605,63.629041,88.500694,4.152408,0.167975,2019-06-15 11:45:44.076327085+00:00,2019-06-15 12:28:30.041616201+00:00,2019-06-15 12:03:37.742481709+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.0,2.473200,0.183374,46.966050,67.893466,97.292764,4.885032,0.125439,2019-06-15 12:43:08.175044537+00:00,2019-06-15 13:49:46.153021574+00:00,2019-06-15 13:12:23.660107613+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.0,2.646600,0.187148,71.732710,103.222325,149.332760,5.468303,0.108405,2019-06-15 14:12:44.651605368+00:00,2019-06-15 15:55:00.262489080+00:00,2019-06-15 14:56:42.113079786+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,6.0,2.153800,0.067659,59.661689,68.152676,77.645740,5.722255,0.085571,2019-06-15 15:19:56.721949339+00:00,2019-06-15 17:04:06.650341034+00:00,2019-06-15 16:05:16.809604168+00:00,0.1147,0.0
2916,740,1,Rasti-Jyry 4,Mila Viksilä,0.0,2.258195,0.182322,40.019664,57.329819,81.908042,4.049051,0.183835,2019-06-15 11:40:01.179843903+00:00,2019-06-15 12:21:54.482538700+00:00,2019-06-15 11:57:19.789157867+00:00,0.0000,0.0
2917,740,2,Rasti-Jyry 4,Tiina Sidensnöre,7.0,2.262600,0.146600,41.226253,54.861778,72.917496,4.726512,0.118278,2019-06-15 12:29:46.613021135+00:00,2019-06-15 13:23:09.210375071+00:00,2019-06-15 12:52:58.078700542+00:00,0.0000,0.0


In [40]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [41]:
for_web.shape

(6728, 17)

In [43]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean", "count"]).round(2)


personal_median       personal_estimate_interval      
                    mean count                       mean count
num_runs                                                       
0.0                81.78  1646                      65.06  1646
1.0                80.72  1021                      61.79  1021
2.0                81.43   730                      58.37   730
3.0                78.10   697                      56.07   697
4.0                76.82   557                      32.71   557
5.0                75.57   566                      35.23   566
6.0                72.99   551                      33.59   551
7.0                71.34   571                      32.77   571
8.0                71.37   386                      32.31   386
9.0                84.05     2                      36.43     2
10.0               62.41     1                      16.53     1

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,
